# MapToPoster - Batch Generation

Génération automatique de posters cartographiques pour les **100 villes les plus visitées au monde** avec les **17 thèmes** disponibles.

**Total: 100 villes × 17 thèmes = 1700 posters**

Format: A4 (8.27×11.69 inches) en PDF

## 1. Installation des dépendances

In [ ]:
!pip install -q osmnx==2.0.7 matplotlib==3.10.8 geopandas==1.1.2 geopy==2.4.1 lat-lon-parser tqdm

## 2. Clone du repository

In [ ]:
import os

REPO_URL = "https://github.com/msgnoki/mapposter.git"
REPO_DIR = "/content/maptoposter"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    print(f"Repo déjà cloné dans {REPO_DIR}")

os.chdir(REPO_DIR)
print(f"Working directory: {os.getcwd()}")

## 3. Téléchargement des Land Polygons OSM (~870MB)

In [ ]:
LAND_POLYGONS_DIR = "data/land_polygons"
LAND_POLYGONS_SHP = f"{LAND_POLYGONS_DIR}/land-polygons-split-4326/land_polygons.shp"

if not os.path.exists(LAND_POLYGONS_SHP):
    os.makedirs(LAND_POLYGONS_DIR, exist_ok=True)
    print("T\u00e9l\u00e9chargement des land polygons OSM (~870MB)...")
    !wget -q --show-progress https://osmdata.openstreetmap.de/download/land-polygons-split-4326.zip -O {LAND_POLYGONS_DIR}/land-polygons.zip
    print("Extraction...")
    !cd {LAND_POLYGONS_DIR} && unzip -q land-polygons.zip && rm land-polygons.zip
    print("\u2713 Land polygons pr\u00eats!")
else:
    print(f"\u2713 Land polygons d\u00e9j\u00e0 pr\u00e9sents: {LAND_POLYGONS_SHP}")

## 4. Configuration

In [ ]:
import json

# Liste des 100 villes les plus visitées au monde
CITIES = [
  {"rank": 1, "city": "Hong Kong", "country": "Hong Kong"},
  {"rank": 2, "city": "Bangkok", "country": "Thailand"},
  {"rank": 3, "city": "London", "country": "United Kingdom"},
  {"rank": 4, "city": "Macau", "country": "Macau"},
  {"rank": 5, "city": "Singapore", "country": "Singapore"},
  {"rank": 6, "city": "Paris", "country": "France"},
  {"rank": 7, "city": "Dubai", "country": "United Arab Emirates"},
  {"rank": 8, "city": "New York City", "country": "United States"},
  {"rank": 9, "city": "Kuala Lumpur", "country": "Malaysia"},
  {"rank": 10, "city": "Istanbul", "country": "Turkey"},
  {"rank": 11, "city": "Delhi", "country": "India"},
  {"rank": 12, "city": "Antalya", "country": "Turkey"},
  {"rank": 13, "city": "Shenzhen", "country": "China"},
  {"rank": 14, "city": "Mumbai", "country": "India"},
  {"rank": 15, "city": "Phuket", "country": "Thailand"},
  {"rank": 16, "city": "Rome", "country": "Italy"},
  {"rank": 17, "city": "Tokyo", "country": "Japan"},
  {"rank": 18, "city": "Pattaya", "country": "Thailand"},
  {"rank": 19, "city": "Taipei", "country": "Taiwan"},
  {"rank": 20, "city": "Mecca", "country": "Saudi Arabia"},
  {"rank": 21, "city": "Guangzhou", "country": "China"},
  {"rank": 22, "city": "Prague", "country": "Czech Republic"},
  {"rank": 23, "city": "Medina", "country": "Saudi Arabia"},
  {"rank": 24, "city": "Seoul", "country": "South Korea"},
  {"rank": 25, "city": "Amsterdam", "country": "Netherlands"},
  {"rank": 26, "city": "Agra", "country": "India"},
  {"rank": 27, "city": "Miami", "country": "United States"},
  {"rank": 28, "city": "Osaka", "country": "Japan"},
  {"rank": 29, "city": "Los Angeles", "country": "United States"},
  {"rank": 30, "city": "Shanghai", "country": "China"},
  {"rank": 31, "city": "Ho Chi Minh City", "country": "Vietnam"},
  {"rank": 32, "city": "Denpasar", "country": "Indonesia"},
  {"rank": 33, "city": "Barcelona", "country": "Spain"},
  {"rank": 34, "city": "Las Vegas", "country": "United States"},
  {"rank": 35, "city": "Milan", "country": "Italy"},
  {"rank": 36, "city": "Chennai", "country": "India"},
  {"rank": 37, "city": "Vienna", "country": "Austria"},
  {"rank": 38, "city": "Johor Bahru", "country": "Malaysia"},
  {"rank": 39, "city": "Jaipur", "country": "India"},
  {"rank": 40, "city": "Canc\u00fan", "country": "Mexico"},
  {"rank": 41, "city": "Berlin", "country": "Germany"},
  {"rank": 42, "city": "Cairo", "country": "Egypt"},
  {"rank": 43, "city": "Athens", "country": "Greece"},
  {"rank": 44, "city": "Orlando", "country": "United States"},
  {"rank": 45, "city": "Moscow", "country": "Russia"},
  {"rank": 46, "city": "Venice", "country": "Italy"},
  {"rank": 47, "city": "Madrid", "country": "Spain"},
  {"rank": 48, "city": "Ha Long", "country": "Vietnam"},
  {"rank": 49, "city": "Riyadh", "country": "Saudi Arabia"},
  {"rank": 50, "city": "Dublin", "country": "Ireland"},
  {"rank": 51, "city": "Florence", "country": "Italy"},
  {"rank": 52, "city": "Hanoi", "country": "Vietnam"},
  {"rank": 53, "city": "Toronto", "country": "Canada"},
  {"rank": 54, "city": "Johannesburg", "country": "South Africa"},
  {"rank": 55, "city": "Sydney", "country": "Australia"},
  {"rank": 56, "city": "Munich", "country": "Germany"},
  {"rank": 57, "city": "Jakarta", "country": "Indonesia"},
  {"rank": 58, "city": "Beijing", "country": "China"},
  {"rank": 59, "city": "Saint Petersburg", "country": "Russia"},
  {"rank": 60, "city": "Brussels", "country": "Belgium"},
  {"rank": 61, "city": "Jerusalem", "country": "Israel"},
  {"rank": 62, "city": "Budapest", "country": "Hungary"},
  {"rank": 63, "city": "Lisbon", "country": "Portugal"},
  {"rank": 64, "city": "Dammam", "country": "Saudi Arabia"},
  {"rank": 65, "city": "Penang Island", "country": "Malaysia"},
  {"rank": 66, "city": "Heraklion", "country": "Greece"},
  {"rank": 67, "city": "Kyoto", "country": "Japan"},
  {"rank": 68, "city": "Zhuhai", "country": "China"},
  {"rank": 69, "city": "Vancouver", "country": "Canada"},
  {"rank": 70, "city": "Chiang Mai", "country": "Thailand"},
  {"rank": 71, "city": "Copenhagen", "country": "Denmark"},
  {"rank": 72, "city": "San Francisco", "country": "United States"},
  {"rank": 73, "city": "Melbourne", "country": "Australia"},
  {"rank": 74, "city": "Krakow", "country": "Poland"},
  {"rank": 75, "city": "Marrakesh", "country": "Morocco"},
  {"rank": 76, "city": "Kolkata", "country": "India"},
  {"rank": 77, "city": "Cebu City", "country": "Philippines"},
  {"rank": 78, "city": "Auckland", "country": "New Zealand"},
  {"rank": 79, "city": "Tel Aviv", "country": "Israel"},
  {"rank": 80, "city": "Guilin", "country": "China"},
  {"rank": 81, "city": "Honolulu", "country": "United States"},
  {"rank": 82, "city": "Hurghada", "country": "Egypt"},
  {"rank": 83, "city": "Warsaw", "country": "Poland"},
  {"rank": 84, "city": "Mu\u011fla", "country": "Turkey"},
  {"rank": 85, "city": "Buenos Aires", "country": "Argentina"},
  {"rank": 86, "city": "Chiba", "country": "Japan"},
  {"rank": 87, "city": "Frankfurt", "country": "Germany"},
  {"rank": 88, "city": "Stockholm", "country": "Sweden"},
  {"rank": 89, "city": "Lima", "country": "Peru"},
  {"rank": 90, "city": "Da Nang", "country": "Vietnam"},
  {"rank": 91, "city": "Batam", "country": "Indonesia"},
  {"rank": 92, "city": "Nice", "country": "France"},
  {"rank": 93, "city": "Fukuoka", "country": "Japan"},
  {"rank": 94, "city": "Abu Dhabi", "country": "United Arab Emirates"},
  {"rank": 95, "city": "Jeju", "country": "South Korea"},
  {"rank": 96, "city": "Porto", "country": "Portugal"},
  {"rank": 97, "city": "Rhodes", "country": "Greece"},
  {"rank": 98, "city": "Rio de Janeiro", "country": "Brazil"},
  {"rank": 99, "city": "Krabi", "country": "Thailand"},
  {"rank": 100, "city": "Bangalore", "country": "India"}
]

# Mapping pays -> continent
CONTINENT_MAP = {
    'China': 'Asia', 'Hong Kong': 'Asia', 'Macau': 'Asia', 'Thailand': 'Asia',
    'Singapore': 'Asia', 'Malaysia': 'Asia', 'Indonesia': 'Asia', 'Vietnam': 'Asia',
    'Japan': 'Asia', 'South Korea': 'Asia', 'Taiwan': 'Asia', 'India': 'Asia',
    'Philippines': 'Asia', 'Israel': 'Asia', 'Saudi Arabia': 'Asia',
    'United Arab Emirates': 'Asia', 'Turkey': 'Asia',
    'France': 'Europe', 'United Kingdom': 'Europe', 'Spain': 'Europe',
    'Italy': 'Europe', 'Germany': 'Europe', 'Netherlands': 'Europe',
    'Austria': 'Europe', 'Belgium': 'Europe', 'Czech Republic': 'Europe',
    'Hungary': 'Europe', 'Portugal': 'Europe', 'Greece': 'Europe',
    'Denmark': 'Europe', 'Sweden': 'Europe', 'Poland': 'Europe',
    'Russia': 'Europe', 'Ireland': 'Europe',
    'United States': 'North_America', 'Canada': 'North_America',
    'Mexico': 'North_America',
    'Brazil': 'South_America', 'Argentina': 'South_America',
    'Peru': 'South_America',
    'Egypt': 'Africa', 'Morocco': 'Africa', 'South Africa': 'Africa',
    'Australia': 'Oceania', 'New Zealand': 'Oceania',
}

# 17 th\u00e8mes disponibles
THEMES = [
    'autumn', 'blueprint', 'contrast_zones', 'copper_patina', 'emerald',
    'forest', 'gradient_roads', 'japanese_ink', 'midnight_blue',
    'monochrome_blue', 'neon_cyberpunk', 'noir', 'ocean',
    'pastel_dream', 'sunset', 'terracotta', 'warm_beige'
]

# --- CONFIGURATION ---
START_INDEX = 0       # Index de d\u00e9part (0 = Hong Kong)
END_INDEX = 100       # Index de fin (100 = toutes les villes)
FORMAT_WIDTH = 8.27   # A4 width in inches
FORMAT_HEIGHT = 11.69 # A4 height in inches
OUTPUT_FORMAT = 'pdf'

selected_cities = CITIES[START_INDEX:END_INDEX]
total = len(selected_cities) * len(THEMES)
print(f"\u2705 Configuration:")
print(f"   Villes: {len(selected_cities)} (index {START_INDEX} \u00e0 {END_INDEX})")
print(f"   Th\u00e8mes: {len(THEMES)}")
print(f"   Total posters: {total}")
print(f"   Format: A4 ({FORMAT_WIDTH}\u00d7{FORMAT_HEIGHT}\") en {OUTPUT_FORMAT}")

## 5. Monter Google Drive (sauvegarde automatique)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Dossier de sauvegarde sur Google Drive
DRIVE_OUTPUT = '/content/drive/MyDrive/MapToPoster'
os.makedirs(DRIVE_OUTPUT, exist_ok=True)
print(f"\u2713 Sauvegarde vers: {DRIVE_OUTPUT}")

## 6. G\u00e9n\u00e9ration batch

In [ ]:
import subprocess
import glob
import time
import shutil
from pathlib import Path
from IPython.display import clear_output

os.chdir(REPO_DIR)

total_generated = 0
total_errors = 0
errors_log = []
start_time = time.time()

for city_idx, city_data in enumerate(selected_cities):
    city = city_data['city']
    country = city_data['country']
    rank = city_data['rank']
    continent = CONTINENT_MAP.get(country, 'Other')
    city_safe = city.replace(' ', '_').replace('/', '_').lower()

    # Cr\u00e9er arbo locale + Drive
    local_dir = f"posters/{continent}/{city_safe}"
    drive_dir = f"{DRIVE_OUTPUT}/{continent}/{city_safe}"
    Path(local_dir).mkdir(parents=True, exist_ok=True)
    Path(drive_dir).mkdir(parents=True, exist_ok=True)

    for theme_idx, theme in enumerate(THEMES):
        poster_num = city_idx * len(THEMES) + theme_idx + 1

        # V\u00e9rifier si d\u00e9j\u00e0 g\u00e9n\u00e9r\u00e9 sur Drive (reprise)
        existing = glob.glob(f"{drive_dir}/*_{theme}_*.{OUTPUT_FORMAT}")
        if existing:
            total_generated += 1
            continue

        # ETA
        elapsed = time.time() - start_time
        if total_generated > 0:
            avg = elapsed / total_generated
            remaining_est = avg * (total - poster_num)
            eta = f"{int(remaining_est//3600)}h{int((remaining_est%3600)//60)}m"
        else:
            eta = "calculating..."

        clear_output(wait=True)
        print(f"\u2550" * 60)
        progress_pct = poster_num / total * 100
        bar_len = 40
        filled = int(bar_len * poster_num / total)
        bar = '\u2588' * filled + '\u2591' * (bar_len - filled)
        print(f"  [{bar}] {progress_pct:.1f}%")
        print(f"  Poster {poster_num}/{total} | ETA: {eta}")
        print(f"  \u2705 {total_generated} g\u00e9n\u00e9r\u00e9s | \u274c {total_errors} erreurs")
        print(f"\u2550" * 60)
        print(f"  #{rank} {city}, {country} ({continent})")
        print(f"  Th\u00e8me: {theme} [{theme_idx+1}/17]")
        print(f"\u2550" * 60)

        cmd = [
            'python3', 'create_map_poster.py',
            '--city', city, '--country', country,
            '--theme', theme,
            '--width', str(FORMAT_WIDTH),
            '--height', str(FORMAT_HEIGHT),
            '--format', OUTPUT_FORMAT
        ]

        try:
            result = subprocess.run(
                cmd, capture_output=True, text=True,
                check=True, timeout=600  # 10 min max par poster
            )

            # D\u00e9placer vers arbo locale
            pdf_files = glob.glob(f'posters/*.{OUTPUT_FORMAT}')
            if pdf_files:
                latest = max(pdf_files, key=os.path.getctime)
                filename = os.path.basename(latest)
                local_path = f"{local_dir}/{filename}"
                os.rename(latest, local_path)

                # Copier vers Google Drive
                shutil.copy2(local_path, f"{drive_dir}/{filename}")

                total_generated += 1
            else:
                total_errors += 1
                errors_log.append(f"{city} - {theme}: fichier non trouv\u00e9")

        except subprocess.TimeoutExpired:
            total_errors += 1
            errors_log.append(f"{city} - {theme}: timeout (>10min)")
        except subprocess.CalledProcessError as e:
            total_errors += 1
            err_msg = e.stderr[-200:] if e.stderr else 'unknown'
            errors_log.append(f"{city} - {theme}: {err_msg}")

elapsed = time.time() - start_time
clear_output(wait=True)
print("\u2550" * 60)
print("\u2728 G\u00c9N\u00c9RATION TERMIN\u00c9E")
print("\u2550" * 60)
print(f"\u23f1  Dur\u00e9e totale: {int(elapsed//3600)}h{int((elapsed%3600)//60)}m{int(elapsed%60)}s")
print(f"\u2705 G\u00e9n\u00e9r\u00e9s: {total_generated}/{total}")
print(f"\u274c Erreurs: {total_errors}")
print(f"\ud83d\udcc1 Sauvegard\u00e9s sur Drive: {DRIVE_OUTPUT}")

## 7. Rapport d'erreurs

In [ ]:
if errors_log:
    print(f"\u274c {len(errors_log)} erreurs:\n")
    for err in errors_log:
        print(f"  - {err}")
else:
    print("\u2705 Aucune erreur!")

## 8. V\u00e9rification de l'arborescence

In [ ]:
print(f"\ud83d\udcc1 Arborescence sur Google Drive:\n")

for continent in sorted(os.listdir(DRIVE_OUTPUT)):
    continent_path = f"{DRIVE_OUTPUT}/{continent}"
    if os.path.isdir(continent_path):
        cities_dirs = sorted(os.listdir(continent_path))
        total_files = sum(
            len(glob.glob(f"{continent_path}/{c}/*.{OUTPUT_FORMAT}"))
            for c in cities_dirs
        )
        print(f"\ud83c\udf0d {continent}/ ({len(cities_dirs)} villes, {total_files} posters)")
        for city_dir in cities_dirs:
            n = len(glob.glob(f"{continent_path}/{city_dir}/*.{OUTPUT_FORMAT}"))
            status = "\u2705" if n == len(THEMES) else f"\u26a0\ufe0f {n}/17"
            print(f"   \u251c\u2500 {city_dir}/ {status}")

## 9. T\u00e9l\u00e9charger en ZIP (optionnel)

In [ ]:
# Cr\u00e9er un ZIP de tous les posters pour t\u00e9l\u00e9chargement direct
!cd posters && zip -r /content/maptoposter_all_posters.zip . -x '*.DS_Store'

from google.colab import files
# D\u00e9commenter pour t\u00e9l\u00e9charger directement:
# files.download('/content/maptoposter_all_posters.zip')
print(f"\n\u2713 ZIP cr\u00e9\u00e9: /content/maptoposter_all_posters.zip")
print(f"  Taille: {os.path.getsize('/content/maptoposter_all_posters.zip') / 1024 / 1024:.1f} MB")